In [3]:
!pip install opencv-python

  Using cached opencv_python-4.5.5.62-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.4 MB)
     |████████████████████████████████| 16.8 MB 5.7 MB/s            


In [8]:
!pip install mediapipe

     |████████████████████████████████| 32.7 MB 6.0 MB/s            
     |████████████████████████████████| 60 kB 3.0 MB/s             
     |████████████████████████████████| 126 kB 3.5 MB/s            
     |████████████████████████████████| 1.1 MB 1.4 MB/s            
     |████████████████████████████████| 66.6 MB 358 kB/s            
     |████████████████████████████████| 11.3 MB 2.9 MB/s            
     |████████████████████████████████| 890 kB 5.3 MB/s            
     |████████████████████████████████| 98 kB 1.3 MB/s            
     |████████████████████████████████| 1.2 MB 5.0 MB/s            
     |████████████████████████████████| 4.3 MB 6.0 MB/s            
     |████████████████████████████████| 40 kB 70 kB/s             


In [65]:
# Import libraries

import cv2
import time
import numpy as np
import mediapipe as mp

In [4]:
mp_drawing = mp.solutions.drawing_utils     # Connecting Keypoints Visuals
mp_pose = mp.solutions.pose                 # Keypoint detection model

In [57]:
def calc_angle(ankle,knee,hip): 
    ''' Arguments:
        ankle, knee, hip -- Values (x,y,z, visibility) of the keypoints
                            ankle, knee and hip.
        
        Returns:
        theta : Angle in degress between the lines joined by coordinates
                (ankle,knee) and (knee,hip)
    '''
    
    # Dropping the z-coordinate
    ankle = np.array([ankle.x, ankle.y])
    knee = np.array([knee.x, knee.y])
    hip = np.array([hip.x, hip.y])
    
    
    # Building the straight lines
    ankle_knee = np.subtract(ankle, knee) 
    knee_hip = np.subtract(knee, hip)
    
    # Calculating the angles between the two straight lines
    angle = np.arccos(np.dot(ankle_knee, knee_hip) / np.multiply(np.linalg.norm(ankle_knee), np.linalg.norm(knee_hip)))
    angle = 180 - 180 * angle / 3.14
    
    
    return np.round(angle,2)

In [130]:
flag = None  # Current position of leg. Either 'straight' or 'bent'
count = 0
start = 0
timer = 0

cap = cv2.VideoCapture('vide.mp4')
pose = mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6)

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret == False:
        break
    
    
    #BGR to RBG
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False    
    
    
    results = pose.process(image) # Make predictions
    try:
        landmarks = results.pose_landmarks.landmark
        hip_left = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
        hip_right = landmarks[mp_pose.PoseLandmark.RIGHT_HIP]
        knee_left = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
        knee_right = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE]
        ankle_left = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]
        ankle_right = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
        right_shoulder_z = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].z
        left_shoulder_z = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].z                        
    except:
        pass
    
    
    if left_shoulder_z < right_shoulder_z: # Body part closer to the camera to be used for phsiotherapy
        cv2.putText(frame, 'Monitoring Left Leg ||',(10,20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (240,240,240), 1)
        angle = calc_angle(ankle_left, knee_left, hip_left)
        
    else:
        cv2.putText(frame, 'Monitoring Right Leg ||',(10,20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (240,240,240), 1)
        angle = calc_angle(ankle_right, knee_right, hip_right)
 
        
    # Visualize angle in the opencv window
    cv2.putText(frame, str(angle), 
                tuple(np.multiply([knee_left.x, knee_left.y], [640,480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (240,255,255),1)
    

    # Straight leg condition
    if angle > 140:              
        if timer >= 8: # If patient had bent its knee for more than 8 sec
            count = count + 1 # Increase the rep counter
            timer = 0 # Reset the timer
            flag = 'straight' # Leg is straight
        elif timer < 8 and flag == 'bent': # Leg straighened before 8 seconds
            cv2.putText(image, 'Please Keep Your Knee Bent',
                        (0, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                        (0,0,255), 2)
            timer = 0 # Timer reset without Rep count increament
            start = time.time()
    
    
    # Bent leg condition
    elif angle < 140:
        if flag == 'straight':
            start = time.time()
            flag = 'bent'
        timer = time.time() - start
                
    
    # Illustratrations on OpenCV Window
    cv2.rectangle(frame, (0,0), (640,60), (245,117,16), -1)
    cv2.putText(frame, 'Reps = ' + str(count), (10,50),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1)
    cv2.putText(frame, "Timer = " + str(round(timer))  , (450, 20),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5,(220,220,220,1))    
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    
    cv2.imshow('Video', frame)    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x559ddfb5ed60) is not the object's thread (0x559dde72ce80).
Cannot move to target thread (0x559ddfb5ed60)

QObject::moveToThread: Current thread (0x559ddfb5ed60) is not the object's thread (0x559dde72ce80).
Cannot move to target thread (0x559ddfb5ed60)

QObject::moveToThread: Current thread (0x559ddfb5ed60) is not the object's thread (0x559dde72ce80).
Cannot move to target thread (0x559ddfb5ed60)

QObject::moveToThread: Current thread (0x559ddfb5ed60) is not the object's thread (0x559dde72ce80).
Cannot move to target thread (0x559ddfb5ed60)

QObject::moveToThread: Current thread (0x559ddfb5ed60) is not the object's thread (0x559dde72ce80).
Cannot move to target thread (0x559ddfb5ed60)

QObject::moveToThread: Current thread (0x559ddfb5ed60) is not the object's thread (0x559dde72ce80).
Cannot move to target thread (0x559ddfb5ed60)

QObject::moveToThread: Current thread (0x559ddfb5ed60) is not the object's thread (0x559dde72ce80).
Cannot move to tar

In [128]:
!pip freeze > requirements.txt